### 方案一

In [ ]:
import paddlehub as hub

senta = hub.Module(name="senta_lstm")

In [ ]:
test_text = ["这家餐厅很好吃", "这部电影真的很差劲"]

In [ ]:
results = senta.sentiment_classify(texts=test_text)

In [ ]:
print(results)

### 方案二

In [ ]:
!pip install paddlepaddle-gpu -U
!pip install paddlehub==2.1.0
!hub install ernie_skep_sentiment_analysis==1.0.0

In [ ]:
from tqdm import tqdm
import numpy as np
Labels = []
Texts = []
with open('data.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for i in tqdm(range(len(lines))):
        if i%2 == 0:
            str1 = lines[i].rstrip()
        else:
            str2 = lines[i].rstrip()
            if len(str2) != 0:
                label = [int(j) for j in str2.split()]
                Labels.append(label)
                Texts.append(str1)
print(len(Texts))

In [ ]:
import paddlehub as hub
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Load ernie_skep_sentiment_analysis module.
module = hub.Module(name="ernie_skep_sentiment_analysis")

# Predict sentiment label
# test_texts = ['你不是不聪明，而是不认真', '虽然小明很努力，但是他还是没有考100分']
results = module.predict_sentiment(Texts, use_gpu=True)

In [ ]:
print(results[:10])

In [ ]:
r_stat = np.zeros([3,125])
for i,r in tqdm(enumerate(results)):
    labels = Labels[i]
    for j in labels:
        r_stat[0,j] += 1
        r_stat[2,j] += r['positive_probs']
        if r['sentiment_label'] == 'positive':
            r_stat[1,j] += 1

r_stat[2,:] = r_stat[2,:]/r_stat[0,:]
r_stat[1,:] = r_stat[1,:]/r_stat[0,:]

#保存对表情情感分析的结果
np.save("emoji_sentiment.npy",r_stat)

In [ ]:
ids = [0,7,8,9,10,13,16,18,19,21,22,23,24,28,33,38,42]
for i in ids:
    print(i,r_stat[1,i])